In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import os

In [2]:
training_data = pd.read_csv('../../../../data/numerai_training_data.csv')
training_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,...,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.50
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,...,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,...,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.25
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,...,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.25
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,...,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75


In [3]:
tournament_data = pd.read_csv('../../../../data/numerai_tournament_data.csv')
tournament_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.5,...,0.75,0.75,1.00,0.75,0.50,0.5,1.0,0.00,0.00,0.25
1,n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.0,...,0.50,0.50,0.75,1.00,0.75,0.5,0.5,0.50,0.50,0.50
2,n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.0,...,0.00,0.00,0.50,0.25,0.00,0.0,0.5,0.50,0.00,1.00
3,n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.0,...,0.00,0.00,0.00,0.25,0.00,0.0,0.0,0.25,0.25,0.50
4,n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.5,...,0.50,0.75,0.00,0.00,0.75,1.0,0.0,0.25,1.00,0.75


In [4]:
tournament_data_for_valid = tournament_data.dropna()
(len(tournament_data_for_valid) / len(tournament_data)) * 100

7.959429606658748

In [5]:
TARGET_NAME = [x for x in training_data.columns if 'target' in x][0]
PREDICTION_NAME = 'prediction'
#feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]
features_df = pd.read_csv('../../../../analyse/feature-selection/feature_selection.csv', index_col=0)
features_df = features_df.loc[features_df['COUNT'] > 1]
feature_cols = features_df.index.tolist()
print(len(feature_cols))
feature_cols

76


['feature_intelligence1',
 'feature_intelligence11',
 'feature_charisma85',
 'feature_strength10',
 'feature_dexterity11',
 'feature_constitution2',
 'feature_constitution31',
 'feature_constitution63',
 'feature_charisma36',
 'feature_constitution104',
 'feature_intelligence8',
 'feature_wisdom14',
 'feature_wisdom34',
 'feature_wisdom35',
 'feature_intelligence4',
 'feature_wisdom8',
 'feature_wisdom10',
 'feature_dexterity6',
 'feature_strength31',
 'feature_strength19',
 'feature_wisdom37',
 'feature_charisma83',
 'feature_charisma74',
 'feature_wisdom46',
 'feature_charisma50',
 'feature_wisdom13',
 'feature_charisma58',
 'feature_constitution55',
 'feature_wisdom7',
 'feature_strength12',
 'feature_dexterity8',
 'feature_charisma57',
 'feature_charisma10',
 'feature_constitution12',
 'feature_wisdom32',
 'feature_charisma34',
 'feature_charisma7',
 'feature_dexterity2',
 'feature_dexterity5',
 'feature_charisma77',
 'feature_dexterity12',
 'feature_intelligence10',
 'feature_cons

In [6]:
def dataframe_cast_to_float32(df, features):
    typemap = {}
    for f in features:
        typemap[f] = 'float32'
    return df.astype(typemap)

training_data = dataframe_cast_to_float32(training_data, feature_cols)
tournament_data = dataframe_cast_to_float32(tournament_data, feature_cols)

In [7]:
predictions = []
for itr in range(30):
    model = XGBRegressor(
        max_depth=5, 
        learning_rate=0.01, 
        n_estimators=2000, 
        n_jobs=-1, 
        colsample_bytree=0.1, 
        seed=itr, 
        gpu_id=0,
        tree_method='gpu_hist')
    model.fit(training_data[feature_cols], training_data[TARGET_NAME])
    predictions.append(model.predict(tournament_data[feature_cols]))

In [8]:
prediction_df = tournament_data["id"].to_frame()
for itr in range(len(predictions)):
    prediction_df[f"prediction_{itr}"] = predictions[itr]
prediction_df.head()

#len(prediction_df) == len(tournament_data)

,id,prediction_0,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction_8,...,prediction_20,prediction_21,prediction_22,prediction_23,prediction_24,prediction_25,prediction_26,prediction_27,prediction_28,prediction_29
0,n0003aa52cab36c2,0.499407,0.501766,0.501525,0.502632,0.499646,0.504450,0.499279,0.504337,0.498630,...,0.502548,0.503761,0.498381,0.500824,0.503319,0.503241,0.500866,0.502479,0.501545,0.502460
1,n000920ed083903f,0.489599,0.491942,0.490739,0.492491,0.491616,0.489273,0.494059,0.489592,0.495199,...,0.493623,0.493292,0.490414,0.491897,0.493836,0.489942,0.492265,0.489790,0.492247,0.493547
2,n0038e640522c4a6,0.497869,0.497796,0.498683,0.497968,0.495932,0.498364,0.495596,0.500125,0.499097,...,0.495977,0.499309,0.498524,0.498136,0.496949,0.496609,0.499484,0.498229,0.499848,0.500398
3,n004ac94a87dc54b,0.506138,0.506081,0.504661,0.505923,0.505166,0.506680,0.506383,0.504308,0.505991,...,0.504056,0.504314,0.505136,0.505688,0.504305,0.505639,0.504090,0.507999,0.505219,0.507080
4,n0052fe97ea0c05f,0.486887,0.487004,0.486879,0.486139,0.486800,0.486468,0.487938,0.488415,0.488040,...,0.489069,0.487017,0.488409,0.488599,0.486458,0.488334,0.486395,0.487895,0.485845,0.486202


In [9]:
valid_idx = tournament_data['target'].isna() == False

# Submissions are scored by spearman correlation
def correlation(predictions, targets):
    ranked_preds = predictions.rank(pct=True, method="first")
    return np.corrcoef(ranked_preds, targets)[0, 1]

# convenience method for scoring
def score(df):
    return correlation(df[PREDICTION_NAME], df[TARGET_NAME])

scores = []

for i in range(len(predictions)):
    scores.append(correlation(prediction_df[valid_idx][f'prediction_{i}'], tournament_data[valid_idx][TARGET_NAME]))
scores

[0.02025907971500448,
 0.020148985105791623,
 0.020567855190531774,
 0.020693076438475455,
 0.020301092100069328,
 0.020639705040431233,
 0.02088101476413906,
 0.020356609107598973,
 0.020842231918382494,
 0.02012982281873745,
 0.020036740804495093,
 0.020732802976806596,
 0.021122641501597372,
 0.02003568973958782,
 0.021261294905887775,
 0.02088058017156041,
 0.020117394573288736,
 0.020745497815917908,
 0.02057271715673143,
 0.02087043164894812,
 0.021249312480162808,
 0.020731160139289814,
 0.021422472412722368,
 0.02007936170082205,
 0.021158577347497258,
 0.021794335461756513,
 0.020623890075643466,
 0.02078612293411344,
 0.021070666148008534,
 0.02071034827843184]

In [10]:
sum(scores)/len(scores)

0.020694050349081044

In [11]:
pred_col_list = prediction_df.columns[prediction_df.columns.str.startswith('prediction_')]
pred_col_list

Index(['prediction_0', 'prediction_1', 'prediction_2', 'prediction_3',
       'prediction_4', 'prediction_5', 'prediction_6', 'prediction_7',
       'prediction_8', 'prediction_9', 'prediction_10', 'prediction_11',
       'prediction_12', 'prediction_13', 'prediction_14', 'prediction_15',
       'prediction_16', 'prediction_17', 'prediction_18', 'prediction_19',
       'prediction_20', 'prediction_21', 'prediction_22', 'prediction_23',
       'prediction_24', 'prediction_25', 'prediction_26', 'prediction_27',
       'prediction_28', 'prediction_29'],
      dtype='object')

In [12]:
prediction_df[PREDICTION_NAME] = prediction_df[pred_col_list].mean(axis=1)
prediction_df.head()

,id,prediction_0,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction_8,...,prediction_21,prediction_22,prediction_23,prediction_24,prediction_25,prediction_26,prediction_27,prediction_28,prediction_29,prediction
0,n0003aa52cab36c2,0.499407,0.501766,0.501525,0.502632,0.499646,0.504450,0.499279,0.504337,0.498630,...,0.503761,0.498381,0.500824,0.503319,0.503241,0.500866,0.502479,0.501545,0.502460,0.501680
1,n000920ed083903f,0.489599,0.491942,0.490739,0.492491,0.491616,0.489273,0.494059,0.489592,0.495199,...,0.493292,0.490414,0.491897,0.493836,0.489942,0.492265,0.489790,0.492247,0.493547,0.492134
2,n0038e640522c4a6,0.497869,0.497796,0.498683,0.497968,0.495932,0.498364,0.495596,0.500125,0.499097,...,0.499309,0.498524,0.498136,0.496949,0.496609,0.499484,0.498229,0.499848,0.500398,0.498390
3,n004ac94a87dc54b,0.506138,0.506081,0.504661,0.505923,0.505166,0.506680,0.506383,0.504308,0.505991,...,0.504314,0.505136,0.505688,0.504305,0.505639,0.504090,0.507999,0.505219,0.507080,0.505622
4,n0052fe97ea0c05f,0.486887,0.487004,0.486879,0.486139,0.486800,0.486468,0.487938,0.488415,0.488040,...,0.487017,0.488409,0.488599,0.486458,0.488334,0.486395,0.487895,0.485845,0.486202,0.487399


In [13]:
correlation(prediction_df[PREDICTION_NAME], tournament_data[TARGET_NAME])

nan

In [14]:
prediction_df[['id', 'prediction']].to_csv('./submit2.csv', index=False)

In [15]:
len(prediction_df) == len(tournament_data)

True